In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from PyPDF2 import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [3]:
pushover_user= os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = 'https://api.pushover.net/1/messages.json'

print(pushover_token, pushover_user, pushover_url)

a57a8qb3f8p2v9gxphr4mmcwydke53 uqarb7c23tfobhha7jj4jbiast85ob https://api.pushover.net/1/messages.json


In [8]:
def push(message):
    print(f"Pushing message: {message}")
    data = {
        'token': pushover_token,
        'user': pushover_user,
        'message': message,
        'title': 'AI Agent'
    }
    response = requests.post(pushover_url, data=data)
    print(response.json())


In [10]:
push('Hey! This is another message from my applications!')

Pushing message: Hey! This is another message from my applications!
{'status': 1, 'request': '8ee50a12-8bdb-4a73-bc1a-4cf825c1ea81'}


In [ ]:
def record_user_details(name, email, notes):
    push(f"Recording interest from {name} with {email} and notes {notes}")
    return { 'recorded': 'Ok'}

def record_unknown_question(question):
    push(f"Recording {question} that I could not answer")
    return { 'recorded': 'Ok'}